In [0]:
import numpy as np
import pandas as pd
import keras
from sklearn.utils import shuffle
from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import *
import cv2
from keras.utils import np_utils
import math 
from keras.optimizers import SGD,Adam 
import os
from zipfile import ZipFile
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint, LearningRateScheduler,  ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
def inception_A(x):
  conv3x3 = Conv2D(64,(1,1),strides = (1,1),padding = 'same',activation = 'relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv3x3 = Conv2D(96,(3,3),strides = (1,1),padding = 'same',activation = 'relu')(conv3x3)
  
  conv5x5 = Conv2D(64,(1,1),strides = (1,1),padding = 'same',activation = 'relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv5x5 = Conv2D(96,(3,3),strides = (1,1),padding = 'same',activation = 'relu')(conv5x5)
  conv5x5 = Conv2D(96,(3,3),strides = (1,1),padding = 'same',activation = 'relu')(conv5x5)

  conv1x1 = Conv2D(96,(1,1),strides = (1,1),padding = 'same',activation = 'relu',kernel_initializer = kernel_init,bias_initializer = bias_init)(x)
  
  avg_pool = AvgPool2D((3,3),strides  = (1,1),padding = 'same')(x)
  avg_pool = Conv2D(96,(1,1),strides = (1,1),padding = 'same',activation = 'relu')(avg_pool)
  
  output = concatenate([avg_pool,conv1x1,conv3x3,conv5x5],axis=3)
  return output

  

In [0]:
def inception_B(x):
  avg_pool = AvgPool2D((3,3),strides  = (1,1),padding = 'same')(x)
  avg_pool = Conv2D(128,(1,1),strides = (1,1),padding = 'same',activation = 'relu')(avg_pool)
  
  conv1x1 = Conv2D(384,(1,1),strides = (1,1),padding = 'same',activation = 'relu',kernel_initializer = kernel_init,bias_initializer = bias_init)(x)
  
  conv3x3 = Conv2D(192,(1,1),strides = (1,1),padding = 'same',activation = 'relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv3x3 = Conv2D(224,(1,7),strides = (1,1),padding = 'same',activation = 'relu')(conv3x3)
  conv3x3 = Conv2D(256,(7,1),strides = (1,1),padding = 'same',activation = 'relu')(conv3x3)
  
  conv5x5 = Conv2D(192,(1,1),strides = (1,1),padding = 'same',activation = 'relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv5x5 = Conv2D(192,(1,7),strides = (1,1),padding = 'same',activation = 'relu')(conv5x5)
  conv5x5 = Conv2D(224,(7,1),strid
                   
                   .es = (1,1),padding = 'same',activation = 'relu')(conv5x5) 
  conv5x5 = Conv2D(224,(1,7),strides = (1,1),padding = 'same',activation = 'relu')(conv5x5)
  conv5x5 = Conv2D(256,(7,1),strides = (1,1),padding = 'same',activation = 'relu')(conv5x5)

   
  output = concatenate([avg_pool,conv1x1,conv3x3,conv5x5],axis=3)
  return output



In [0]:
def inception_C(x):
  avg_pool = AvgPool2D((3,3),strides  = (1,1),padding = 'same')(x)
  avg_pool = Conv2D(256,(1,1),strides = (1,1),padding = 'same',activation = 'relu')(avg_pool)
  
  conv1x1 = Conv2D(256,(1,1),strides = (1,1),padding = 'same',activation = 'relu',kernel_initializer = kernel_init,bias_initializer = bias_init)(x)

  conv3x3 = Conv2D(384,(1,1),strides = (1,1),padding = 'same',activation = 'relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_1_3x3 = Conv2D(256,(1,3),strides = (1,1),padding = 'same',activation = 'relu')(conv3x3)
  conv_2_3x3 = Conv2D(256,(3,1),strides = (1,1),padding = 'same',activation = 'relu')(conv3x3)
  
  conv5x5 = Conv2D(384,(1,1),strides = (1,1),padding = 'same',activation = 'relu',kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv5x5 = Conv2D(448,(1,3),strides = (1,1),padding = 'same',activation = 'relu')(conv5x5)
  conv5x5 = Conv2D(512,(3,1),strides = (1,1),padding = 'same',activation = 'relu')(conv5x5) 
  conv_1_5x5 = Conv2D(256,(1,3),strides = (1,1),padding = 'same',activation = 'relu')(conv5x5)
  conv_2_5x5 = Conv2D(256,(3,1),strides = (1,1),padding = 'same',activation = 'relu')(conv5x5)

  output = concatenate([avg_pool,conv1x1,conv_1_3x3,conv_2_3x3,conv_1_5x5,conv_2_5x5],axis=3)
  
  return output
  
  
  

In [0]:
def reduction_A(x):
  max_pool = MaxPool2D((3,3),strides = (2,2))(x)
  
  conv3x3 = Conv2D(384,(3,3),strides = (2,2),activation = 'relu',kernel_initializer = kernel_init,bias_initializer = bias_init)(x)
  
  conv5x5 = Conv2D(192,(1,1),strides = (1,1),activation = 'relu',padding = 'same',kernel_initializer = kernel_init,bias_initializer = bias_init)(x)
  conv5x5 = Conv2D(224,(3,3),strides = (1,1),activation = 'relu',padding = 'same')(conv5x5)
  conv5x5 = Conv2D(256,(3,3),strides = (2,2),activation = 'relu')(conv5x5)
  
  output = concatenate([max_pool,conv3x3,conv5x5],axis = 3)
                  
  return output                 
                   

In [0]:
def reduction_B(x):
  max_pool = MaxPool2D((3,3),strides = (2,2))(x)
    
  conv3x3 = Conv2D(192,(1,1),strides = (1,1),padding = 'same',activation = 'relu',kernel_initializer = kernel_init,bias_initializer = bias_init)(x)
  conv3x3 = Conv2D(192,(3,3),strides = (2,2),activation = 'relu')(conv3x3)
  
  conv5x5 = Conv2D(256,(1,1),strides = (1,1),activation = 'relu',padding = 'same',kernel_initializer = kernel_init,bias_initializer = bias_init)(x)
  conv5x5 = Conv2D(256,(1,7),strides = (1,1),activation = 'relu',padding = 'same')(conv5x5)
  conv5x5 = Conv2D(320,(7,1),strides = (1,1),activation = 'relu',padding = 'same')(conv5x5)
  conv5x5 = Conv2D(320,(3,3),strides = (2,2),activation = 'relu')(conv5x5)
  
  output = concatenate([max_pool,conv3x3,conv5x5],axis=3)
  
  return output


In [0]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [0]:
input_layer = Input(shape=(299,299, 3))
x = Conv2D(32,(3,3),strides = (2,2),activation = 'relu',name='conv_1_3x3/2',kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = Conv2D(32,(3,3),strides = (1,1),activation = 'relu',name='conv_2_3x3/1')(x)
x = Conv2D(64,(3,3),strides = (1,1),padding = 'same',activation = 'relu',name='conv_3_3x3/1')(x)

a = MaxPool2D((3,3),strides = (2,2),name = 'max_pool_1_3x3/2')(x)
b = Conv2D(96,(3,3),strides = (2,2),activation = 'relu',name='conv_4_3x3/2')(x)

x = concatenate([a,b], axis=3, name='concatenate1')

a = Conv2D(64,(1,1),strides = (1,1),activation = 'relu',name = 'conv_5_1x1')(x)
a = Conv2D(96,(3,3),strides = (1,1),activation = 'relu',name = 'conv_6_3x3/1')(a)

b = Conv2D(64,(1,1),strides = (1,1),activation = 'relu',name = 'conv_7_1x1')(x)
b = Conv2D(64,(7,1),strides = (1,1),padding = 'same',activation = 'relu',name = 'conv_8_7x1')(b)
b = Conv2D(64,(1,7),strides = (1,1),padding = 'same',activation = 'relu',name = 'conv_9_1x7')(b)
b = Conv2D(96,(3,3),strides = (1,1),activation = 'relu',name = 'conv_10_3x3')(b)

x = concatenate([a,b],axis = 3,name = 'concatenate2')

a = Conv2D(192,(3,3),strides = (2,2),activation = 'relu',name = 'conv_11_3x3/2')(x)

b = MaxPool2D((3,3),strides = (2,2),name = 'max_pool_2_3x3/2')(x)

x = concatenate([a,b],axis=3,name = 'concatenate3')

for i in range(0,4):
  x = inception_A(x)
x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(2, activation='softmax', name='auxilliary_output_1')(x1)


x = reduction_A(x)

for i in range(0,7):
  x = inception_B(x)

x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(2, activation='softmax', name='auxilliary_output_2')(x2)

x = reduction_B(x)

for i in range(0,3):
  x = inception_C(x)

x = GlobalAveragePooling2D(name='avg_pool_1_8x8/1')(x)
x = Dropout(0.8)(x)
x = Dense(2, activation='softmax', name='output')(x)









W0620 14:41:35.321054 139848380815232 nn_ops.py:4224] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0620 14:41:36.364260 139848380815232 nn_ops.py:4224] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [0]:
model = Model(input_layer,[x,x1,x2],name = 'inceptionV4')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv_1_3x3/2 (Conv2D)           (None, 149, 149, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
conv_2_3x3/1 (Conv2D)           (None, 147, 147, 32) 9248        conv_1_3x3/2[0][0]               
__________________________________________________________________________________________________
conv_3_3x3/1 (Conv2D)           (None, 147, 147, 64) 18496       conv_2_3x3/1[0][0]               
__________________________________________________________________________________________________
max_pool_1